In [1]:
import importer
import tensorflow as tf
import pickle
import numpy as np

In [2]:
# import pickle
with open('data.pickle', 'rb') as file_handler:
    data = pickle.load(file_handler)

In [3]:
data

{'filename': array(['1445267970.48394_2.daedalus.boiteameuh.org',
        '1423882957.56155_2.daedalus.boiteameuh.org',
        '1437951776.26567_2.daedalus.boiteameuh.org', ...,
        '1390309358.70628_1.leviathan.boiteameuh.org',
        '1390305199.53275_1.leviathan.boiteameuh.org',
        '1390337634.86743_1.leviathan.boiteameuh.org'], 
       dtype='<U69'),
 'hot-ones': array([[ True,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False],
        [ True, False,  True, ..., False, False, False],
        ..., 
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]], dtype=bool),
 'labels': array([False, False, False, ..., False, False, False], dtype=bool),
 'messageid': array(['<15080aedee2.38a2.2757308@ismtpd-075>',
        '<6e95ee8d709029b0dd088a2657ece068@async.facebook.com>',
        '<AD.2D.23363.E2765B55@rs-ord-mta07-

In [4]:
(data['labels'][:,np.newaxis]).shape

(76300, 1)

In [5]:
class SALearner:
    def __init__(self, data, learning_rate):
        # extract data
        # TODO rename filename to filenames
        self.filenames = data['filename']
        # TODO rename messageid to messageids
        self.messageids = data['messageid']
        # TODO check the names hot-ones
        self.dataset = data['hot-ones'].astype(np.float32)
        # TODO return a real column vector on importer
        self.labels = data['labels'][:,np.newaxis].astype(np.float32)
        # description variables
        self.nb_ind = data['hot-ones'].shape[0]
        self.ind_size = data['hot-ones'].shape[1]
        # Variables learned
        self.tf_weights = None
        self.tf_bias = None
        # data and labels
        self.tf_data = None
        self.tf_labels = None
        # unit result
        self.tf_z = None
        self.tf_h = None
        # Hyperparameters
        self.learning_rate = learning_rate
        
    def init_graph(self):
        tf.reset_default_graph()
        # data
        self.tf_data = tf.placeholder(tf.float32)
        self.tf_labels = tf.placeholder(tf.float32)
        # weights and bias
        self.tf_weights = tf.Variable(tf.truncated_normal([self.ind_size, 1]))
        self.tf_bias = tf.Variable(tf.zeros([1,1]))
        # Compute the unit unit
        self.tf_z = tf.matmul(self.tf_data, self.tf_weights) + self.tf_bias #z.shape -> [ind_size,1]
        self.tf_h = tf.sigmoid(self.tf_z)
        
        #self.tf_cost_function = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.tf_z, labels=self.tf_labels))
        self.tf_log = -tf.log(self.tf_h)
        self.tf_first_part = tf.multiply(self.tf_labels, -tf.log(self.tf_h))
        self.tf_cost_function = tf.reduce_mean(self.tf_labels * -tf.log(tf.clip_by_value(self.tf_h,1e-10, 1)) + \
                                               (1 - self.tf_labels) * -tf.log(tf.clip_by_value(1 - self.tf_h, 1e-10, 1)))
        tf_decision_vector = tf.round(self.tf_h)
        self.tf_correct_prediction = tf.equal(self.tf_labels, tf_decision_vector)
        self.tf_accuracy = tf.reduce_mean(tf.cast(self.tf_correct_prediction, tf.float32)) * 100
        
        # GD
        self.tf_train_step = tf.train.GradientDescentOptimizer(self.learning_rate).minimize(self.tf_cost_function)
        
        self.tf_init = tf.global_variables_initializer()
        
    def learn(self, nb_cycle, info_every_cycle):
        sess = tf.Session()
        sess.run(self.tf_init)
        
        for cycle in range(nb_cycle):
            _, cost, acc, z, data, labels, weights, bias, log, fp, h = sess.run([self.tf_train_step, self.tf_cost_function,
                                                                              self.tf_accuracy, self.tf_z, self.tf_data,
                                                                              self.tf_labels, self.tf_weights,
                                                                              self.tf_bias, self.tf_log,
                                                                              self.tf_first_part, self.tf_h],
                                feed_dict={self.tf_data: self.dataset,
                                           self.tf_labels: self.labels})
            if cycle % info_every_cycle == 0:
                print('-' * 10)
                print('cycle: {}'.format(cycle))
                print('cost: {}'.format(cost))
                print('accuracy: {}%'.format(acc))
                #print('label shape: {}'.format(labels.shape))
                #print('-log(h): {}'.format(log.shape))
                #print('fp: {}'.format(fp.shape))
                print(bias)
                print(weights[0:10,0])
                print(z[0:10,0])
                print(h[0:10,0])
                print(labels[0:10,0])
        

In [ ]:
sa_learner = SALearner(data, 0.1)

In [ ]:
sa_learner.init_graph()
sa_learner.learn(100000,20)

----------
cycle: 0
cost: 1.554047703742981
accuracy: 37.08649444580078%
[[ 0.]]
[ 0.18001024 -0.59290737 -1.64361894  0.58765608 -1.07210898  1.09030843
 -1.80362117  0.56327987 -1.01558781 -0.83570898]
[-1.45066071 -6.71029282  0.20967126  1.33357847  0.47130972 -3.72580409
 -1.90820944 -1.30475938  1.55760324 -1.46886015]
[ 0.18989991  0.00121682  0.5522266   0.79143196  0.61569375  0.02352687
  0.12918214  0.21336511  0.82600915  0.18711592]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
----------
cycle: 20
cost: 0.707513153553009
accuracy: 61.216224670410156%
[[-0.63048053]]
[-0.42529926 -0.59853089 -1.69660652  0.54864019 -1.0733819   0.73163676
 -1.82200861  0.56346357 -1.00764871 -0.85162896]
[-3.14402199 -8.04594707 -1.02976203 -1.09408295 -2.3660512  -5.17887688
 -4.30712891 -3.54588485 -0.23863077 -2.80227685]
[  4.13274802e-02   3.20295221e-04   2.63130248e-01   2.50850201e-01
   8.57983679e-02   5.60276303e-03   1.32930866e-02   2.80344896e-02
   4.40623790e-01   5.72012626e-0